# ud


In [4]:
#NOMENCLATURE REFERENCE
def viterbi_algorithm(S, Π, E, P, Y):
    K = len(S)
    T = len(Y)
    
    # Part I: Initialization
    viterbi_prob = np.zeros((K, T))
    viterbi_path = np.zeros((K, T), dtype=int)
    
    for i in range(K):
        viterbi_prob[i][0] = Π[i] * E[i][Y[0]]
    
    # Part II: Compute Viterbi probabilities and Viterbi path
    for j in range(1, T):
# likelihoo = likelihood_bruteforce(obs, states, start_prob, trans_prob, emit_prob)
# print("Probability of observing the sequence of activities (using brute-force addition):", likelihoo)
        for i in range(K):
            max_prob = 0
            max_path = 0
            
            for α in range(K):
                prob = E[i][Y[j]] * P[α][i] * viterbi_prob[α][j - 1]
                if prob > max_prob:
                    max_prob = prob
                    max_path = α
            
            viterbi_prob[i][j] = max_prob
            viterbi_path[i][j] = max_path
    
    # Part III: Re-tracking the most likely path X
    X = [0] * T
    max_prob = 0
    max_state = 0
    
    for i in range(K):
        if viterbi_prob[i][T - 1] > max_prob:
            max_prob = viterbi_prob[i][T - 1]
            max_state = i
            
    X[T - 1] = S[max_state]
    
    for j in range(T - 2, -1, -1):
        max_state = viterbi_path[max_state][j + 1]
        X[j] = S[max_state]
    
    return X, max_prob

# Define observations, states, and probability matrices
obs = [0,0,2,1,2]  # 0: reading, 1: playing, 2: shopping
states = [0, 1, 2]  # 0: rainy, 1: cloudy, 2: sunny

PiMatrix = np.array([0.35, 0.25, 0.4])
transitionMatrix = np.array([[0.4, 0.2, 0.4],
                       [0.2, 0.5, 0.3],
                       [0.3, 0.6, 0.1]])
emissionMatrix = np.array([[0.8, 0.1, 0.1],
                      [0.2, 0.5, 0.3],
                      [0.4, 0.2, 0.4]])

# Run Viterbi algorithm
X, prob = viterbi_algorithm(states, PiMatrix, emissionMatrix, transitionMatrix, obs)
print("Most likely path:", X)
print("Probability of the most likely path:", prob)

Most likely path: [0, 0, 2, 1, 1]
Probability of the most likely path: 0.0006451200000000002


# ALL IN ONE

In [3]:
import numpy as np

# def likelihood_bruteforce(obs, states, start_prob, trans_prob, emit_prob):
#     T = len(obs)
#     N = len(states)
    
#     likelihood = 0.0
    
#     state_sequences = generate_state_sequences(states, T)
    
#     for sequence in state_sequences:
#         prob = start_prob[sequence[0]] * emit_prob[sequence[0], obs[0]]
#         for t in range(1, T):
#             prob *= trans_prob[sequence[t-1], sequence[t]] * emit_prob[sequence[t], obs[t]]
#         likelihood += prob
    
#     return likelihood

# def generate_state_sequences(states, length):
#     if length == 1:
#         return [[state] for state in states]
#     else:
#         prev_sequences = generate_state_sequences(states, length-1)
#         new_sequences = []
#         for seq in prev_sequences:
#             for state in states:
#                 new_seq = seq + [state]
#                 new_sequences.append(new_seq)
#         return new_sequences


def viterbi(obs, states, PiMatrix, transitionMatrix, emissionMatrix):
    T = len(obs)
    N = len(states)

    # Initialize viterbi matrix and backpointer matrix
    viterbi_mat = np.zeros((N, T))
    prevStateIndex = np.zeros((N, T), dtype=int)
    fwd_prob = np.zeros((N, T))

    # Initialization step
    for s in range(N):
        viterbi_mat[s, 0] = PiMatrix[s] * emissionMatrix[s, obs[0]]
        prevStateIndex[s, 0] = 0
        fwd_prob[s, 0] = PiMatrix[s] * emissionMatrix[s, obs[0]]
        

    # Recursion step
    for t in range(1, T):
        for s in range(N):
            probabilities = viterbi_mat[:, t-1] * transitionMatrix[:, s] * emissionMatrix[s, obs[t]]
            viterbi_mat[s, t] = np.max(probabilities)
            prevStateIndex[s, t] = np.argmax(probabilities)
            fwd_prob[s, t] = np.sum(fwd_prob[:, t-1] * transitionMatrix[:, s]) * emissionMatrix[s, obs[t]]
    

    # Termination step
    BestSeqProb = np.max(viterbi_mat[:, -1])
    FinalStateIndex = np.argmax(viterbi_mat[:, -1])

    # Backtracking to find the best path
    PathArray = [FinalStateIndex]
    for t in range(T - 1, 0, -1):
        FinalStateIndex = prevStateIndex[FinalStateIndex, t]
        PathArray.insert(0, FinalStateIndex)
        
    likelihood = np.sum(fwd_prob[:, -1])
    
    return PathArray, BestSeqProb, likelihood
# Define observations, states, and probability matrices
obs = [0,0,2,1,2]  # 0: reading, 1: playing, 2: shopping
states = [0, 1, 2]  # 0: rainy, 1: cloudy, 2: sunny

PiMatrix = np.array([0.35, 0.25, 0.4])
transitionMatrix = np.array([[0.4, 0.2, 0.4],
                       [0.2, 0.5, 0.3],
                       [0.3, 0.6, 0.1]])
emissionMatrix = np.array([[0.8, 0.1, 0.1],
                      [0.2, 0.5, 0.3],
                      [0.4, 0.2, 0.4]])
a = ["rainy"," cloudy"," sunny"]
# (a) Find the most likely weather sequence for the last 5 days
best_path, probability, likelihood = viterbi(obs, states, PiMatrix, transitionMatrix, emissionMatrix)
weather_sequence = [s+1 for s in best_path]  # Convert states to 1-indexed (1: rainy, 2: cloudy, 3: sunny)
weather_string = [a[i] for i in best_path]
print("Most likely weather sequence:", weather_sequence)
print("Most likely weather sequence:", weather_string)

print("Probability of the given sequence of activities:", probability)

# print("Using forward algorithm:" , likelihood)

# likelihood = likelihood_bruteforce(obs, states, start_prob, trans_prob, emit_prob)
# print("Probability of observing the sequence of activities (using brute-force addition):", likelihood)

Most likely weather sequence: [1, 1, 3, 2, 2]
Most likely weather sequence: ['rainy', 'rainy', ' sunny', ' cloudy', ' cloudy']
Probability of the given sequence of activities: 0.00064512


### Q1 (a & b)

In [3]:
import numpy as np

def viterbi(obs, states, PiMatrix, transitionMatrix, emissionMatrix):
    T = len(obs)
    N = len(states)

    # init
    viterbi_mat = np.zeros((N, T))
    prevStateIndex = np.zeros((N, T), dtype=int)
    fwd_prob = np.zeros((N, T))
    
    for s in range(N):
        viterbi_mat[s, 0] = PiMatrix[s] * emissionMatrix[s, obs[0]]
        prevStateIndex[s, 0] = 0
        fwd_prob[s, 0] = PiMatrix[s] * emissionMatrix[s, obs[0]]
        
    # applying viterbi
    for t in range(1, T):
        for s in range(N):
            probabilities = viterbi_mat[:, t-1] * transitionMatrix[:, s] * emissionMatrix[s, obs[t]]
            viterbi_mat[s, t] = np.max(probabilities)
            prevStateIndex[s, t] = np.argmax(probabilities)
            fwd_prob[s, t] = np.sum(fwd_prob[:, t-1] * transitionMatrix[:, s]) * emissionMatrix[s, obs[t]]
    

    # most likely path's last state
    BestSeqProb = np.max(viterbi_mat[:, -1])
    matrix = []
    for i in range(N):
        if viterbi_mat[i, -1] == BestSeqProb:
            matrix.append(i)
            
    FinalStateIndex = np.argmax(viterbi_mat[:, -1])

    # using our pathArray to get most likely seq
    PathArray = [FinalStateIndex]
    for t in range(T - 1, 0, -1):
        FinalStateIndex = prevStateIndex[FinalStateIndex, t]
        PathArray.insert(0, FinalStateIndex)
        
    likelihood = np.sum(fwd_prob[:, -1])
    
    return PathArray, BestSeqProb, likelihood

# INPUTS
# 0: rainy, 1: cloudy, 2: sunny
obs = [0,0,2,1,2]  
states = [0, 1, 2]

PiMatrix = np.array([0.35, 0.25, 0.4])
transitionMatrix = np.array([[0.4, 0.2, 0.4],
                       [0.2, 0.5, 0.3],
                       [0.3, 0.6, 0.1]])
emissionMatrix = np.array([[0.8, 0.1, 0.1],
                      [0.2, 0.5, 0.3],
                      [0.4, 0.2, 0.4]])
a = ["rainy"," cloudy"," sunny"]
best_path, probability, likelihood = viterbi(obs, states, PiMatrix, transitionMatrix, emissionMatrix)
weather_sequence = [s+1 for s in best_path]  # convert states to 1-indexed (1: rainy, 2: cloudy, 3: sunny)
weather_string = [a[i] for i in best_path]
print("Most likely weather sequence:", weather_sequence)
print("Most likely weather sequence:", weather_string)

print("Probability of the given sequence of activities:", probability)

Most likely weather sequence: [1, 1, 3, 2, 2]
Most likely weather sequence: ['rainy', 'rainy', ' sunny', ' cloudy', ' cloudy']
Probability of the given sequence of activities: 0.00064512


# 2 (a,b)

In [5]:
import numpy as np

def viterbi(obs, states, PiMatrix, transitionMatrix, emissionMatrix):
    T = len(obs)
    N = len(states)
#init
    viterbi_mat = np.zeros((N, T))
    prevStateIndex = np.zeros((N, T), dtype=int)
    forward = np.zeros((N, T))

    for s in range(N):
        viterbi_mat[s, 0] = PiMatrix[s] * emissionMatrix[s, obs[0]]
        prevStateIndex[s, 0] = 0
        forward[s, 0] = PiMatrix[s] * emissionMatrix[s, obs[0]]
        
    # applying viterbi
    for t in range(1, T):
        for s in range(N):
            probabilities = viterbi_mat[:, t-1] * transitionMatrix[:, s] * emissionMatrix[s, obs[t]]
            viterbi_mat[s, t] = np.max(probabilities)
            prevStateIndex[s, t] = np.argmax(probabilities)
            forward[s, t] = np.sum(forward[:, t-1] * transitionMatrix[:, s]) * emissionMatrix[s, obs[t]]  #getting total probability only limited by obs

    # most likely path's last state
    BestSeqProb = np.max(viterbi_mat[:, -1])
    FinalStateIndex = np.argmax(viterbi_mat[:, -1])

    # using our pathArray to get most likely seq
    PathArray = [FinalStateIndex]
    for t in range(T - 1, 0, -1):
        FinalStateIndex = prevStateIndex[FinalStateIndex, t]
        PathArray.insert(0, FinalStateIndex)

    likelihood = np.sum(forward[:, -1])
    
    return PathArray, BestSeqProb, likelihood

# function for better showcase, comfortable input and clarity
def intObsTOString(observed_sequence):
    return [0 if obs == 'up' else 1 for obs in observed_sequence]

# input data and matrices
stringObs = ['up', 'down', 'down', 'down', 'up', 'up', 'up', 'down', 'up', 'down']
obs = intObsTOString(stringObs)
states = [0, 1, 2] # normal, alternate, direct

PiMatrix = np.array([0.2, 0.3, 0.5])
TransMatrix = np.array([[0.35, 0.45, 0.2],
              [0.28, 0.32, 0.4],
              [0.75, 0.1, 0.15]])
emissionMatrix = np.array([[0.1, 0.9], [0.6, 0.4],[0.7, 0.3]])
a = ["normal"," alternate"," direct"]
# calling our function
best_path, probability, likelihood = viterbi(obs, states, PiMatrix, TransMatrix, emissionMatrix)
weather_sequence = [s+1 for s in best_path]  # Convert states to 1-indexed (1: rainy, 2: cloudy, 3: sunny)
weather_string = [a[i] for i in best_path]

print("Most likely control sequence:", weather_sequence)
print("Most likely control sequence:", weather_string)
print("Probability of the given sequence of activities:", probability)


stringObs1 = ['up', 'down', 'down', 'down', 'up', 'up', 'up', 'down', 'up', 'down']
obs1 = intObsTOString(stringObs)
_,_,likelihood1 = viterbi(obs1, states, PiMatrix, TransMatrix, emissionMatrix)

stringObs2 = ['up', 'down', 'down', 'down', 'up', 'up', 'up', 'down', 'up', 'down']
obs2 = intObsTOString(stringObs)
_,_,likelihood2 = viterbi(obs2, states, PiMatrix, TransMatrix, emissionMatrix)

stringObs3 = ['up', 'down', 'down', 'down', 'up', 'up', 'up', 'down', 'up', 'down']
obs3 = intObsTOString(stringObs)
_,_,likelihood3 = viterbi(obs3, states, PiMatrix, TransMatrix, emissionMatrix)

stringObs4 = ['up', 'down', 'down', 'down', 'up', 'up', 'up', 'down', 'up', 'down']
obs4 = intObsTOString(stringObs)
_,_,likelihood4 = viterbi(obs4, states, PiMatrix, TransMatrix, emissionMatrix)

stringObs5 = ['up', 'down', 'down', 'down', 'up', 'up', 'up', 'down', 'up', 'down']
obs5 = intObsTOString(stringObs)
_,_,likelihood5 = viterbi(obs5, states, PiMatrix, TransMatrix, emissionMatrix)

print("\nb.\n(i): ", likelihood1)
print("(ii): ", likelihood2)
print("(iii): ", likelihood3)
print("(iv): ", likelihood4)
print("(v): ", likelihood5)



Most likely control sequence: [3, 1, 1, 1, 2, 2, 3, 1, 3, 1]
Most likely control sequence: [' direct', 'normal', 'normal', 'normal', ' alternate', ' alternate', ' direct', 'normal', ' direct', 'normal']
Probability of the given sequence of activities: 2.1704587195410003e-05

b.
(i):  0.0011761669647830317
(ii):  0.0011761669647830317
(iii):  0.0011761669647830317
(iv):  0.0011761669647830317
(v):  0.0011761669647830317
